In [60]:
import json
import os
import time
import aiohttp

import firebase_admin
from firebase_admin import firestore
from google.cloud.firestore import Client
from rich import print
from tqdm import tqdm
import pyEX as pyex
import yahooquery as yq
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("ALPACA_KEY", "")
api_secret = os.environ.get("ALPACA_SECRET", "")


In [3]:

with open("/Users/peter/Projects/socii/app/socii-development-serviceAccountKey.json", "r") as f:
    service_account_info = json.load(f)

credentials = firebase_admin.credentials.Certificate(service_account_info)

# initialize firebase admin
firebase_app = firebase_admin.initialize_app(
    credentials, options=dict(storageBucket="sociiinvest.appspot.com")
)

client: Client = firestore.client()
iex = pyex.Client(api_token=os.environ.get("IEX_TOKEN", ""))


In [4]:
async def get_broker_api_endpoint(endpoint:str, params:dict):
    async with aiohttp.ClientSession(
        auth=aiohttp.BasicAuth(api_key, api_secret)
    ) as session:
        async with session.get(
            os.environ.get("ALPACA_BASE_URL", "") + endpoint + 
            ("?" + "&".join([f"{k}={v}" for k, v in params.items()]) if params else ""),
        ) as response:
            data = await response.json()
            return data


In [6]:
def read_alpaca_assets_from_file():
    with open("./alpaca/data/assets.json", "r") as f:
        return json.load(f)

alpaca_assets = read_alpaca_assets_from_file()

In [7]:
alpaca_assets_df = pd.DataFrame.from_dict(alpaca_assets)

In [13]:
tsla = yq.Ticker("TSLA")

In [21]:
tsla.esg_scores["TSLA"]["peerGroup"]

'Automobiles'

In [23]:
with open("./alpaca/data/firestore_alpaca_assets.json", "r") as f:
    firestore_alpaca_assets_df = pd.DataFrame.from_dict(json.load(f)).T

In [32]:
firestore_alpaca_assets_df.alpaca.map(lambda x : x['exchange']).value_counts()

NASDAQ    3818
NYSE      2374
AMEX       256
ARCA         6
BATS         1
Name: alpaca, dtype: int64

In [64]:
def batch_by_n(n, iterable):
    return list(zip(*[iter(iterable)] * n))

peer_groups = {}
for batch in tqdm(batch_by_n(20, pd.json_normalize(firestore_alpaca_assets_df.alpaca).query("status == 'active' and exchange == 'NASDAQ'").symbol.to_list())):
    result = yq.Ticker(' '.join(batch)).esg_scores
    for ticker in batch:
        if type(result[ticker]) == dict:
            peer_groups[ticker] = result[ticker]["peerGroup"]
    time.sleep(0.5)

print(peer_groups)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 187/187 [20:20<00:00,  6.53s/it]


In [ ]:
amex_peer_groups = {
    'BCV': 'US CE Convertibles',
    'CET': 'US CE Large Blend',
    'CLM': 'US CE Large Blend',
    'CRF': 'US CE Large Blend',
    'ECF': 'US CE Convertibles',
    'ERH': 'US CE Allocation--50% to 70% Equity',
    'FEN': 'US CE Energy Limited Partnership',
    'GLQ': 'US CE World Large Stock',
    'GLU': 'US CE Utilities',
    'GLV': 'US CE Tactical Allocation',
    'GRF': 'US CE Large Blend',
    'IAF': 'US CE Miscellaneous Region',
    'IMO': 'Oil & Gas Producers',
    'LNG': 'Refiners & Pipelines',
    'NRO': 'US CE Real Estate',
    'UTG': 'US CE Utilities',
    'GLO': 'US CE Tactical Allocation'
}

nyse_peer_group = {
    'MCK': 'Healthcare',
    'GPC': 'Retailing',
    'ABBV': 'Pharmaceuticals',
    'ABC': 'Healthcare',
    'ACN': 'Software & Services',
    'AAP': 'Retailing',
    'ADS': 'Software & Services',
    'AFG': 'Insurance',
    'AFL': 'Insurance',
    'COTY': 'Household Products',
    'DG': 'Retailing',
    'EQT': 'Oil & Gas Producers',
    'T': 'Telecommunication Services',
    'AIG': 'Insurance',
    'AIV': 'Real Estate',
    'AIZ': 'Insurance',
    'AJG': 'Insurance',
    'ALB': 'Chemicals',
    'ALLE': 'Building Products',
    'ALLY': 'Diversified Financials',
    'AME': 'Electrical Equipment',
    'AMG': 'Diversified Financials',
    'AMP': 'Diversified Financials',
    'AMT': 'Real Estate',
    'ANET': 'Technology Hardware',
    'ANTM': 'Healthcare',
    'AOS': 'Machinery',
    'APD': 'Chemicals',
    'APH': 'Technology Hardware',
    'APTV': 'Auto Components',
    'AR': 'Oil & Gas Producers',
    'ARE': 'Real Estate',
    'ARMK': 'Consumer Services',
    'ARW': 'Technology Hardware',
    'AVB': 'Real Estate',
    'AVY': 'Containers & Packaging',
    'AXP': 'Diversified Financials',
    'AXS': 'Insurance',
    'AXTA': 'Chemicals',
    'AYI': 'Electrical Equipment',
    'AZO': 'Retailing',
    'BA': 'Aerospace & Defense',
    'BAC': 'Banks',
    'BAH': 'Software & Services',
    'BAM': 'Diversified Financials',
    'BAX': 'Healthcare',
    'BBY': 'Retailing',
    'BDX': 'Healthcare',
    'BEN': 'Diversified Financials',
    'BEP': 'Utilities',
    'BG': 'Food Products',
    'BHC': 'Pharmaceuticals',
    'BIO': 'Pharmaceuticals',
    'BK': 'Diversified Financials',
    'BLK': 'Diversified Financials',
    'BLL': 'Containers & Packaging',
    'BMY': 'Pharmaceuticals',
    'BNS': 'Banks',
    'BR': 'Software & Services',
    'BRO': 'Insurance',
    'BRX': 'Real Estate',
    'BSX': 'Healthcare',
    'BWA': 'Auto Components',
    'BX': 'Diversified Financials',
    'BXP': 'Real Estate',
    'C': 'Banks',
    'CAG': 'Food Products',
    'CAH': 'Healthcare',
    'CB': 'Insurance',
    'CBRE': 'Real Estate',
    'CCI': 'Real Estate',
    'CCK': 'Containers & Packaging',
    'CCL': 'Consumer Services',
    'CE': 'Chemicals',
    'CF': 'Chemicals',
    'CFG': 'Banks',
    'CFR': 'Banks',
    'CHD': 'Household Products',
    'CHN': 'US CE China Region',
    'CIT': 'Banks',
    'CIXX': 'Diversified Financials',
    'CL': 'Household Products',
    'COF': 'Diversified Financials',
    'COG': 'Oil & Gas Producers',
    'COP': 'Oil & Gas Producers',
    'CPB': 'Food Products',
    'CRM': 'Software & Services',
    'CSL': 'Building Products',
    'CVE': 'Oil & Gas Producers',
    'CVS': 'Healthcare',
    'CVX': 'Oil & Gas Producers',
    'D': 'Utilities',
    'DAL': 'Transportation',
    'DB': 'Banks',
    'DCI': 'Machinery',
    'DE': 'Machinery',
    'DEI': 'Real Estate',
    'DELL': 'Technology Hardware',
    'DFS': 'Diversified Financials',
    'DGX': 'Healthcare',
    'DHI': 'Homebuilders',
    'DHR': 'Pharmaceuticals',
    'DIS': 'Media',
    'DKS': 'Retailing',
    'DLR': 'Real Estate',
    'DOV': 'Machinery',
    'DPZ': 'Consumer Services',
    'DRI': 'Consumer Services',
    'DTE': 'Utilities',
    'DUK': 'Utilities',
    'DVA': 'Healthcare',
    'DVN': 'Oil & Gas Producers',
    'DXC': 'Software & Services',
    'ECL': 'Chemicals',
    'ED': 'Utilities',
    'EDU': 'Consumer Services',
    'EFX': 'Commercial Services',
    'EIX': 'Utilities',
    'EL': 'Household Products',
    'EMN': 'Chemicals',
    'ENB': 'Refiners & Pipelines',
    'EOG': 'Oil & Gas Producers',
    'EQR': 'Real Estate',
    'ES': 'Utilities',
    'ESS': 'Real Estate',
    'ET': 'Refiners & Pipelines',
    'ETN': 'Electrical Equipment',
    'ETR': 'Utilities',
    'EW': 'Healthcare',
    'EXP': 'Construction Materials',
    'EXR': 'Real Estate',
    'F': 'Automobiles',
    'FBHS': 'Building Products',
    'FDS': 'Software & Services',
    'FCX': 'Diversified Metals',
    'FE': 'Utilities',
    'FIS': 'Software & Services',
    'FL': 'Retailing',
    'FLR': 'Construction & Engineering',
    'FLS': 'Machinery',
    'FLT': 'Software & Services',
    'FMC': 'Chemicals',
    'FRC': 'Banks',
    'FRT': 'Real Estate',
    'FTS': 'Utilities',
    'FTV': 'Industrial Conglomerates',
    'GD': 'Aerospace & Defense',
    'GGG': 'Machinery',
    'GIB': 'Software & Services',
    'GIL': 'Textiles & Apparel',
    'GIS': 'Food Products',
    'GL': 'Insurance',
    'GOLD': 'Precious Metals',
    'GPN': 'Software & Services',
    'GPS': 'Retailing',
    'GS': 'Diversified Financials',
    'GWW': 'Traders & Distributors',
    'H': 'Consumer Services',
    'HAL': 'Energy Services',
    'HCA': 'Healthcare',
    'HD': 'Retailing',
    'HFC': 'Refiners & Pipelines',
    'HIG': 'Insurance',
    'HLF': 'Household Products',
    'HLT': 'Consumer Services',
    'HOG': 'Automobiles',
    'HP': 'Energy Services',
    'HPE': 'Technology Hardware',
    'HPQ': 'Technology Hardware',
    'HRL': 'Food Products',
    'HSY': 'Food Products',
    'HUBB': 'Electrical Equipment',
    'HUM': 'Healthcare',
    'HUN': 'Chemicals',
    'IBM': 'Software & Services',
    'ICE': 'Diversified Financials',
    'ICL': 'Chemicals',
    'IEX': 'Machinery',
    'IFF': 'Chemicals',
    'IGI': 'US CE Corporate Bond',
    'IGT': 'Consumer Services',
    'INFO': 'Commercial Services',
    'INGR': 'Food Products',
    'IPG': 'Media',
    'IRM': 'Real Estate',
    'IT': 'Commercial Services',
    'ITW': 'Machinery',
    'J': 'Construction & Engineering',
    'JCI': 'Building Products',
    'JEF': 'Diversified Financials',
    'JLL': 'Real Estate',
    'JNPR': 'Technology Hardware',
    'JPS': 'US CE Preferred Stock',
    'JWN': 'Retailing',
    'K': 'Food Products',
    'KAR': 'Commercial Services',
    'KEY': 'Banks',
    'KEYS': 'Technology Hardware',
    'KIM': 'Real Estate',
    'KMB': 'Household Products',
    'KMI': 'Refiners & Pipelines',
    'KMX': 'Retailing',
    'KO': 'Food Products',
    'KR': 'Food Retailers',
    'KSS': 'Retailing',
    'KSU': 'Transportation',
    'L': 'Diversified Financials',
    'LAZ': 'Diversified Financials',
    'LDOS': 'Software & Services',
    'LEG': 'Industrial Conglomerates',
    'LEN': 'Homebuilders',
    'LH': 'Healthcare',
    'LII': 'Consumer Durables',
    'LLY': 'Pharmaceuticals',
    'LNC': 'Insurance',
    'LOW': 'Retailing',
    'LUMN': 'Telecommunication Services',
    'LUV': 'Transportation',
    'LVS': 'Consumer Services',
    'LYB': 'Chemicals',
    'M': 'Retailing',
    'MA': 'Software & Services',
    'MAA': 'Real Estate',
    'MAC': 'Real Estate',
    'MAN': 'Commercial Services',
    'MAS': 'Building Products',
    'MCD': 'Consumer Services',
    'MCO': 'Diversified Financials',
    'MD': 'Healthcare',
    'MDU': 'Construction Materials',
    'MET': 'Insurance',
    'MFC': 'Insurance',
    'MGA': 'Auto Components',
    'MHK': 'Consumer Durables',
    'MKL': 'Insurance',
    'MLM': 'Construction Materials',
    'MMC': 'Insurance',
    'MMM': 'Industrial Conglomerates',
    'MO': 'Food Products',
    'MOS': 'Chemicals',
    'MPC': 'Refiners & Pipelines',
    'MRK': 'Pharmaceuticals',
    'MRO': 'Oil & Gas Producers',
    'MS': 'Diversified Financials',
    'MSCI': 'Diversified Financials',
    'MSI': 'Technology Hardware',
    'MSM': 'Traders & Distributors',
    'MTB': 'Banks',
    'MTD': 'Pharmaceuticals',
    'NCLH': 'Consumer Services',
    'NEE': 'Utilities',
    'NEM': 'Precious Metals',
    'NLSN': 'Commercial Services',
    'NLY': 'Real Estate',
    'NOC': 'Aerospace & Defense',
    'NOV': 'Energy Services',
    'NOW': 'Software & Services',
    'NRG': 'Utilities',
    'NSC': 'Transportation',
    'NTR': 'Chemicals',
    'NUE': 'Steel',
    'NVR': 'Homebuilders',
    'NYCB': 'Banks',
    'O': 'Real Estate',
    'OC': 'Building Products',
    'OHI': 'Real Estate',
    'OMC': 'Media',
    'ORCL': 'Software & Services',
    'ORI': 'Insurance',
    'OXY': 'Oil & Gas Producers',
    'PANW': 'Software & Services',
    'PCG': 'Utilities',
    'PEAK': 'Real Estate',
    'PEG': 'Utilities',
    'PFE': 'Pharmaceuticals',
    'PG': 'Household Products',
    'PGR': 'Insurance',
    'PHM': 'Homebuilders',
    'PII': 'Consumer Durables',
    'PKG': 'Containers & Packaging',
    'PKI': 'Pharmaceuticals',
    'PLD': 'Real Estate',
    'PM': 'Food Products',
    'PNC': 'Banks',
    'PNR': 'Machinery',
    'PNW': 'Utilities',
    'PPG': 'Chemicals',
    'PPL': 'Utilities',
    'PRGO': 'Pharmaceuticals',
    'PRU': 'Insurance',
    'PSA': 'Real Estate',
    'PSX': 'Refiners & Pipelines',
    'PVH': 'Textiles & Apparel',
    'PWR': 'Construction & Engineering',
    'PXD': 'Oil & Gas Producers',
    'QGEN': 'Pharmaceuticals',
    'QSR': 'Consumer Services',
    'RACE': 'Automobiles',
    'RAD': 'Food Retailers',
    'RCI': 'Telecommunication Services',
    'RCL': 'Consumer Services',
    'RE': 'Insurance',
    'RF': 'Banks',
    'RGA': 'Insurance',
    'RHI': 'Commercial Services',
    'RIG': 'Energy Services',
    'RJF': 'Diversified Financials',
    'RL': 'Textiles & Apparel',
    'RMD': 'Healthcare',
    'RNR': 'Insurance',
    'ROK': 'Electrical Equipment',
    'ROL': 'Commercial Services',
    'ROP': 'Software & Services',
    'RPM': 'Chemicals',
    'RRC': 'Oil & Gas Producers',
    'RS': 'Traders & Distributors',
    'RSG': 'Commercial Services',
    'RY': 'Banks',
    'SCCO': 'Diversified Metals',
    'SCHW': 'Diversified Financials',
    'SCI': 'Consumer Services',
    'SEE': 'Containers & Packaging',
    'SHW': 'Chemicals',
    'SIG': 'Retailing',
    'SITC': 'Real Estate',
    'SIX': 'Consumer Services',
    'SJM': 'Food Products',
    'SJR': 'Media',
    'SLB': 'Energy Services',
    'SLF': 'Insurance',
    'SLG': 'Real Estate',
    'SMG': 'Chemicals',
    'SNA': 'Machinery',
    'SNV': 'Banks',
    'SO': 'Utilities',
    'SON': 'Containers & Packaging',
    'SPG': 'Real Estate',
    'SQ': 'Software & Services',
    'SRE': 'Utilities',
    'ST': 'Technology Hardware',
    'STZ': 'Food Products',
    'SU': 'Oil & Gas Producers',
    'SWK': 'Machinery',
    'SYF': 'Diversified Financials',
    'SYK': 'Healthcare',
    'TAL': 'Consumer Services',
    'TAP': 'Food Products',
    'TD': 'Banks',
    'TDG': 'Aerospace & Defense',
    'TEL': 'Technology Hardware',
    'TFX': 'Healthcare',
    'TGNA': 'Media',
    'TGT': 'Retailing',
    'THO': 'Automobiles',
    'TJX': 'Retailing',
    'TMO': 'Pharmaceuticals',
    'TNL': 'Consumer Services',
    'TOL': 'Homebuilders',
    'TPR': 'Textiles & Apparel',
    'TRGP': 'Refiners & Pipelines',
    'TRI': 'Media',
    'TRP': 'Refiners & Pipelines',
    'TRQ': 'Diversified Metals',
    'TRV': 'Insurance',
    'TT': 'Building Products',
    'TTC': 'Machinery',
    'TU': 'Telecommunication Services',
    'TWTR': 'Software & Services',
    'UDR': 'Real Estate',
    'UGI': 'Refiners & Pipelines',
    'UHS': 'Healthcare',
    'UNH': 'Healthcare',
    'UNM': 'Insurance',
    'UPS': 'Transportation',
    'URI': 'Traders & Distributors',
    'USB': 'Banks',
    'V': 'Software & Services',
    'UNP': 'Transportation',
    'VER': 'Real Estate',
    'BAP': 'Banks',
    'VIPS': 'Retailing',
    'VLO': 'Refiners & Pipelines',
    'VMC': 'Construction Materials',
    'VMW': 'Software & Services',
    'VNO': 'Real Estate',
    'VOYA': 'Insurance',
    'VTR': 'Real Estate',
    'VZ': 'Telecommunication Services',
    'WAB': 'Machinery',
    'WAT': 'Pharmaceuticals',
    'WCN': 'Commercial Services',
    'WELL': 'Real Estate',
    'WFC': 'Banks',
    'WLK': 'Chemicals',
    'WM': 'Commercial Services',
    'WMB': 'Refiners & Pipelines',
    'WPC': 'Real Estate',
    'WPM': 'Precious Metals',
    'WRB': 'Insurance',
    'WTRG': 'Utilities',
    'WU': 'Software & Services',
    'WY': 'Paper & Forestry',
    'X': 'Steel',
    'XOM': 'Oil & Gas Producers',
    'Y': 'Insurance',
    'FDX': 'Transportation',
    'MKC': 'Food Products',
    'WEC': 'Utilities',
    'YUM': 'Consumer Services',
    'ZBH': 'Healthcare',
    'CNP': 'Utilities',
    'ZTS': 'Pharmaceuticals',
    'CP': 'Transportation',
    'AER': 'Traders & Distributors',
    'AES': 'Utilities',
    'ALV': 'Auto Components',
    'ASH': 'Chemicals',
    'ATO': 'Utilities',
    'CAT': 'Machinery',
    'CPRI': 'Textiles & Apparel',
    'AWK': 'Utilities',
    'BCE': 'Telecommunication Services',
    'CI': 'Healthcare',
    'CLR': 'Oil & Gas Producers',
    'CLX': 'Household Products',
    'CM': 'Banks',
    'CMA': 'Banks',
    'CMG': 'Consumer Services',
    'CMI': 'Machinery',
    'CNA': 'Insurance',
    'CNC': 'Healthcare',
    'CNHI': 'Machinery',
    'CNI': 'Transportation',
    'CNQ': 'Oil & Gas Producers',
    'COO': 'Healthcare',
    'CPG': 'Oil & Gas Producers',
    'CPT': 'Real Estate',
    'DD': 'Chemicals',
    'IVZ': 'Diversified Financials',
    'FNF': 'Insurance',
    'FNV': 'Precious Metals',
    'HES': 'Oil & Gas Producers',
    'IP': 'Paper & Forestry',
    'IQV': 'Pharmaceuticals',
    'NEU': 'Chemicals',
    'NKE': 'Textiles & Apparel',
    'OKE': 'Refiners & Pipelines',
    'PBA': 'Refiners & Pipelines',
    'SPR': 'Aerospace & Defense',
    'STT': 'Diversified Financials',
    'WHR': 'Consumer Durables',
    'XYL': 'Machinery',
    'TECK': 'Diversified Metals',
    'EMR': 'Machinery',
    'AEE': 'Utilities',
    'ABT': 'Healthcare',
    'ADM': 'Food Products',
    'DRE': 'Real Estate',
    'LMT': 'Aerospace & Defense',
    'LEA': 'Auto Components',
    'MGM': 'Consumer Services',
    'PH': 'Machinery',
    'ACM': 'Construction & Engineering',
    'CMS': 'Utilities',
    'GM': 'Automobiles',
    'GLW': 'Technology Hardware',
    'TSN': 'Food Products',
    'UBS': 'Diversified Financials',
    'TXT': 'Aerospace & Defense',
    'UAA': 'Retailing',
    'VFC': 'Textiles & Apparel',
    'OGE': 'Utilities',
    'NI': 'Utilities',
    'HBI': 'Textiles & Apparel',
    'SYY': 'Food Retailers',
    'AEM': 'Precious Metals',
    'ALL': 'Insurance',
    'JPM': 'Banks',
    'MDT': 'Healthcare',
    'NCR': 'Technology Hardware',
    'XEC': 'Oil & Gas Producers',
    'BMO': 'Banks'
}

nasdaq_peer_groups = {
    'ADI': 'Semiconductors',
    'ADP': 'Software & Services',
    'ADSK': 'Software & Services',
    'AEP': 'Utilities',
    'BOKF': 'Banks',
    'AGNC': 'Real Estate',
    'AKAM': 'Software & Services',
    'ALKS': 'Pharmaceuticals',
    'AMGN': 'Pharmaceuticals',
    'AMZN': 'Retailing',
    'ANSS': 'Software & Services',
    'APA': 'Oil & Gas Producers',
    'ATVI': 'Software & Services',
    'AVGO': 'Semiconductors',
    'AVT': 'Technology Hardware',
    'BBBY': 'Retailing',
    'BIDU': 'Software & Services',
    'BIIB': 'Pharmaceuticals',
    'BMRN': 'Pharmaceuticals',
    'CBSH': 'Banks',
    'CDK': 'Software & Services',
    'CDW': 'Technology Hardware',
    'CERN': 'Healthcare',
    'CERS': 'EAA Fund US Large-Cap Blend Equity',
    'CHK': 'Oil & Gas Producers',
    'CHKP': 'Software & Services',
    'CHRW': 'Transportation',
    'CHTR': 'Telecommunication Services',
    'CINF': 'Insurance',
    'COST': 'Food Retailers',
    'CSCO': 'Technology Hardware',
    'CSGP': 'Software & Services',
    'CTSH': 'Software & Services',
    'DISCA': 'Media',
    'DISH': 'Media',
    'DLTR': 'Retailing',
    'DXCM': 'Healthcare',
    'EBAY': 'Software & Services',
    'EQIX': 'Real Estate',
    'EWBC': 'Banks',
    'EXC': 'Utilities',
    'EXPD': 'Transportation',
    'EXPE': 'Software & Services',
    'FAST': 'Traders & Distributors',
    'FB': 'Software & Services',
    'FFIV': 'Software & Services',
    'FISV': 'Software & Services',
    'FITB': 'Banks',
    'FLEX': 'Technology Hardware',
    'FTNT': 'Software & Services',
    'GILD': 'Pharmaceuticals',
    'GLPI': 'Real Estate',
    'GNTX': 'Auto Components',
    'GOOGL': 'Software & Services',
    'GRMN': 'Consumer Durables',
    'GT': 'Auto Components',
    'HAS': 'Consumer Durables',
    'HBAN': 'Banks',
    'HOLX': 'Healthcare',
    'HON': 'Industrial Conglomerates',
    'HSIC': 'Healthcare',
    'HST': 'Real Estate',
    'IAC': 'Software & Services',
    'IDXX': 'Healthcare',
    'ILMN': 'Pharmaceuticals',
    'INCY': 'Pharmaceuticals',
    'IPGP': 'Technology Hardware',
    'ISRG': 'Healthcare',
    'JAZZ': 'Pharmaceuticals',
    'JBHT': 'Transportation',
    'JKHY': 'Software & Services',
    'KHC': 'Food Products',
    'KLAC': 'Semiconductors',
    'LAMR': 'Media',
    'LBRDK': 'Media',
    'LBTYA': 'Telecommunication Services',
    'LECO': 'Machinery',
    'LKQ': 'Retailing',
    'LNT': 'Utilities',
    'LRCX': 'Semiconductors',
    'LSXMK': 'Media',
    'LULU': 'Textiles & Apparel',
    'MAR': 'Consumer Services',
    'MAT': 'Consumer Durables',
    'MCHP': 'Semiconductors',
    'MDLZ': 'Food Products',
    'MIDD': 'Machinery',
    'MLCO': 'Consumer Services',
    'MNST': 'Food Products',
    'MRVL': 'Semiconductors',
    'MSFT': 'Software & Services',
    'MU': 'Semiconductors',
    'NDAQ': 'Diversified Financials',
    'NDSN': 'Machinery',
    'NTAP': 'Technology Hardware',
    'NTRS': 'Diversified Financials',
    'NVDA': 'Semiconductors',
    'NWL': 'Consumer Durables',
    'NWSA': 'Media',
    'NXPI': 'Semiconductors',
    'ON': 'Semiconductors',
    'ORLY': 'Retailing',
    'OTEX': 'Software & Services',
    'PCAR': 'Machinery',
    'PEP': 'Food Products',
    'PFG': 'Insurance',
    'PTC': 'Software & Services',
    'PYPL': 'Software & Services',
    'QCOM': 'Semiconductors',
    'QRVO': 'Semiconductors',
    'REG': 'Real Estate',
    'RMRM': 'US CE Real Estate',
    'SABR': 'Software & Services',
    'SATS': 'Technology Hardware',
    'SBAC': 'Real Estate',
    'SBNY': 'Banks',
    'SBUX': 'Consumer Services',
    'SEIC': 'Diversified Financials',
    'SGEN': 'Pharmaceuticals',
    'SIVB': 'Banks',
    'SLM': 'Diversified Financials',
    'SNPS': 'Software & Services',
    'SPLK': 'Software & Services',
    'SRCL': 'Commercial Services',
    'SSNC': 'Software & Services',
    'STLD': 'Steel',
    'STX': 'Technology Hardware',
    'SVC': 'Real Estate',
    'SWKS': 'Semiconductors',
    'TCOM': 'Software & Services',
    'TER': 'Semiconductors',
    'TMUS': 'Telecommunication Services',
    'TRIP': 'Software & Services',
    'TRMB': 'Technology Hardware',
    'TROW': 'Diversified Financials',
    'TSCO': 'Retailing',
    'TSLA': 'Automobiles',
    'UHAL': 'Transportation',
    'ULTA': 'Retailing',
    'UTHR': 'Pharmaceuticals',
    'VIACA': 'Media',
    'VRSK': 'Commercial Services',
    'VRSN': 'Software & Services',
    'VRTX': 'Pharmaceuticals',
    'WBA': 'Food Retailers',
    'WFRD': 'Energy Services',
    'WLTW': 'Insurance',
    'WYNN': 'Consumer Services',
    'XEL': 'Utilities',
    'XLNX': 'Semiconductors',
    'XRAY': 'Healthcare',
    'CME': 'Diversified Financials',
    'YNDX': 'Software & Services',
    'Z': 'Software & Services',
    'AAPL': 'Technology Hardware',
    'BKNG': 'Retailing',
    'CMCSA': 'Telecommunication Services',
    'COMM': 'Technology Hardware',
    'CPRT': 'Commercial Services',
    'CSX': 'Transportation',
    'CTAS': 'Commercial Services',
    'CTXS': 'Software & Services',
    'INTC': 'Semiconductors',
    'INTU': 'Software & Services',
    'IONS': 'Pharmaceuticals',
    'NLOK': 'Software & Services',
    'PACW': 'Banks',
    'PAYX': 'Software & Services',
    'PBCT': 'Banks',
    'REGN': 'Pharmaceuticals',
    'ACGL': 'Insurance',
    'CDNS': 'Software & Services',
    'EA': 'Software & Services',
    'ADBE': 'Software & Services',
    'DOX': 'Software & Services',
    'ERIE': 'Insurance',
    'AMAT': 'Semiconductors',
    'AAL': 'Transportation',
    'TXN': 'Semiconductors',
    'UAL': 'Transportation',
    'SIRI': 'Media',
    'WDAY': 'Software & Services',
    'WDC': 'Technology Hardware'
}